# Codificación Huffman


Para la codificación Huffman necesitamos poder representar el árbol, para esto se puede implementar una clase `Nodo` en base al cual se puede construir un árbol enlazando unos nodos con otros. Este es un nodo genérico que se puede utilizar para cualquier árbol binario.

In [ ]:
class Nodo:
    def __init__(self, v):
        self.valor = v
        self.izquierda = None
        self.derecha = None
        
    def esHoja(self):
        if (self.izquierda ==None and self.derecha == None):
            return True
        return False

Podemos probarlo construyendo un árbol cualquiera

In [ ]:
raiz = Nodo("Padre")
raiz.izquierda = Nodo("hijo1")
raiz.derecha = Nodo("hijo2")
raiz.izquierda.esHoja()

True

Pero el nodo que necesitamos para la codificación debe incluir además del valor la frecuencia con la que aparece ese símbolo en la cadena que se codifica. Para esto derivamos una clase de `Nodo` llamada `NodoHuffman`. Implementando un método que permita fusionar nodos para agregar 0s o 1s. Para esto sobrecargamos el método `__add__` que nos permite utilizar el operador `+` con los nodos.

In [ ]:
class NodoHuffman(Nodo):
    def __init__(self, v, f):
        Nodo.__init__(self, v)
        self.frecuencia = f
        
    def __add__(self, n):
        return NodoHuffman(self.valor + n.valor, self.frecuencia+n.frecuencia);

Vamos a insntanciar algunos nodos Huffman y probar como se fusionan. El nodo resultante tiene como valor la concatenación de los valores de los sumandos y la frecuencia es la suma de las frecuencias de los sumandos.

In [ ]:
nh1 = NodoHuffman('A', 3)
nh2 = NodoHuffman('B', 5)
nh3 = nh1 + nh2
print(nh3.valor, nh3.frecuencia)

AB 8


El codificador se implementa con la clase `Huffman` que contiene los métodos que permiten construir el árbol en base a los símbolos de la cadena a codificar, y las frecuencias de estos.

In [ ]:
class Huffman:
    def __init__(self, mensaje):
        self.codigos = {}  #Diccionario con los códigos calculados
        self.simbolos = None  #lista de NodoHuffman(s) de símbolos y frecuencias
        self.arbol = None
        self.contarFrecuencias(mensaje)
        self.generaArbol()
        self.generahuffman(self.arbol)

    def contarFrecuencias(self, texto):
        '''
        Crea una lista de NodoHuffman(s) cada uno con su símbolo y su frecuencia

        Parameters
        ----------
        texto : el string a codificar

        Returns
        -------
        None.

        '''
        letras = [l for l in texto]
        unicas = set(letras)   #Poniendolo en un conjunto se eliminan los duplicados
        self.simbolos = [ NodoHuffman(l, texto.count(l)) for l in unicas]

    def frecuencia(self, n):
        return n.frecuencia

    def generaArbol(self):
        '''
        Acomoda los nodos en un arbol binario en base a las frecuencias 

        Returns
        -------
        None.

        '''
        frecuencias = self.simbolos
        frecuencias.sort(key=self.frecuencia)  #ordena los nodos por frecuencias 
        while(len(frecuencias)>1):
            Padre = frecuencias[0]+frecuencias[1]
            Padre.izquierda = frecuencias[0]
            Padre.derecha = frecuencias[1]
            frecuencias = [Padre]+frecuencias[2:]
            frecuencias.sort(key=self.frecuencia)
        self.arbol = frecuencias[0]  #asigna el primer nodo como raiz del arbol

    def generahuffman(self, arbol, codigo=''):
        '''
        Gerera los códigos Huffman para cada nodo del árbol, recorriendo el 
        arbol de forma recursiva. Los códigos se registran en el diccionario
        de códigos.

        Parameters
        ----------
        arbol : Recibel el nodo que hace de raiz del subarbol.
        codigo : Recibe el código calculado hasta antes de la recursión

        Returns
        -------
        codigo : El código huffman calculado hasta el nodo.

        '''
        if arbol.esHoja():
            self.codigos.setdefault(arbol.valor, codigo)
            return codigo
        self.generahuffman(arbol.derecha, codigo+'0')  # 0 + subarbol derecho
        self.generahuffman(arbol.izquierda, codigo + '1') # 1 + subarbol izquierdo
        
    def __str__(self):
        codstring = ""
        for c in self.codigos:
            codstring += "{} : {} \n".format(c,self.codigos[c])
        return codstring
    
    def codifica(self):
        '''
        Devuelve la cadena codificada con la codificación Huffman contenida en el diccionario self.codigos
        
        Returns
        -------
        codigo : El código huffman calculado hasta el nodo. P.ejm. 
            "000010101010101000001010100111111101010101010101"
        '''
        pass

Utilizamos la clase `Huffman` para codificar una cadena, la clase está programada para que haga todos los cálculos al momento de instanciarse, debido a que se calcula un código Huffman para cada cadena, el codificador se instancia por única vez para cada secuencia.

In [ ]:
H = Huffman('ESTE ES UN MENSAJE DE PRUEBA PARA DEMOSTRAR LA CODIFICACION HUFFMAN')
codigos = H.codigos
print(codigos)

{'M': '00000', 'O': '00001', 'I': '00010', 'C': '00011', ' ': '001', 'S': '0100', 'J': '010100', 'L': '010101', 'H': '010110', 'B': '010111', 'N': '0110', 'R': '0111', 'A': '100', 'E': '101', 'P': '11000', 'T': '11001', 'F': '1101', 'U': '1110', 'D': '1111'}


Este diccionario será utilizado luego para generar la secuencia codificada. pero si queremos verlo de forma ordenada lo podemos hacer gracias a que hemos sobrecargado el método `__str__()` y obtener una versión amigable de los códigos.

In [ ]:
print(H)

M : 00000 
O : 00001 
I : 00010 
C : 00011 
  : 001 
S : 0100 
J : 010100 
L : 010101 
H : 010110 
B : 010111 
N : 0110 
R : 0111 
A : 100 
E : 101 
P : 11000 
T : 11001 
F : 1101 
U : 1110 
D : 1111 



## Ejercicio
1. Implemente el método `codifica` de la clase `Huffman`.
2. Implemente una función `decodificar(codigos, seqcodificada)` (fuera de la clase `Huffman`) que reciba un diccionario con los códigos de cada símbolo y una secuencia codificada  y en base a estas reconstruya el mensaje original.
3. Modifique la definición del método `__str__` de la clase `Huffman` para que muestre los códigos de forma que se aprecie la forma del árbol. (Sugerencia: utilice recursividad)
4. Descargue el libro [Don Quijote de la Mancha](https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt) y comprímalo utilizándo codificación Huffman. Luego descomprímalo y verifique que no ha perdido datos.